# Huggett (1993)

Load all necessary packages:

In [19]:
using Parameters
using QuantEcon
using Gadfly
using LinearAlgebra
using SparseArrays

In [3]:
mutable struct Agent
  σ::Float64
  β::Float64
  r::Float64
  w::Float64
  e::Vector{Float64} #vector of productivities
  π::Vector{Float64}
  agrid::Vector{Float64}
end;

In [5]:
"""
Utility function for agent when consumption is a real number
"""
function utility(agent::Agent,c::Real)
  σ = agent.σ
  if c > 0
    if σ == 1
      return log(c)
    else
      return c^(1-σ)/(1-σ)
    end
  else
    return -Inf
  end
end;

In [7]:
"""
Utility function when consumption is a vector
"""
function utility(agent::Agent,cvec)
  u(c) = utility(agent,c)
  return u.(cvec)
end;

In [8]:
"""
  iterateBellman(agent::Agent,Vcont)

Iterates on the bellman equation using continuation value function Vcont
"""
function iterateBellman(agent::Agent,Vcont)
  @unpack σ,agrid,π,w,e,r,β = agent #unpack parameters of agent
  u(c) = utility(agent,c)#shorthand for utility
  Na = length(agrid)
  S = length(e)

  EVcont = Vcont*π #precompute expected value

  #solve for each state
  #preallocate space for V and a_policy
  V = similar(Vcont)
  a_policy = similar(Vcont,Int)
  for s  in 1:S
    for ia_ in 1:Na
      cvec = e[s]*w + (1+r)*agrid[ia_] - agrid #possible consumption values for each asset position
      obj = u(cvec) + β * EVcont #objective for each asset position

      V[ia_,s],a_policy[ia_,s] = findmax(obj) #choose maximum asset position
    end
  end
  return V,a_policy
end;

In [9]:
"""
  iterateBellman_alt(agent::Agent,Vcont)

Iterates on the bellman equation using continuation value function Vcont.  Exploits
that a_policy in increasing in a and that the value function is concave.
"""
function iterateBellman_alt(agent::Agent,Vcont)
  @unpack σ,agrid,π,w,e,r,β = agent
  u(c) = utility(agent,c)
  Na = length(agrid)
  S = length(e)
  EVcont = Vcont*π #precompute expected value

  EVcont = Vcont*π #precompute expected value

  #solve for each state
  #preallocate space for V and a_policy
  V = similar(Vcont) #creates V the same size as Vcont
  a_policy = similar(Vcont,Int) #note a_policy needs to be integers
  for s  in 1:S
    aprev = 1 #stores for the last ia_ the optimal policy
    for ia_ in 1:Na
      maxobj = -Inf #stores the maxobj up to this point
      for ia in aprev:Na #start iterating at a_policy[ia_-1,s]
        c = w*e[s] + (1+r)*agrid[ia_] - agrid[ia] #get consumption for this ia
        obj = u(c) + β * EVcont[ia] #get objective
        if obj >= maxobj #check if increasing ia improves utility
          V[ia_,s],a_policy[ia_,s] = obj,ia #if so store values
          maxobj = obj#and update maxobj
        else
          break #if it decreases utility end for loop, we have found our max
        end
      end
      aprev = a_policy[ia_,s]#store the max in aprev for next ia_
    end
  end
  return V,a_policy
end;

In [10]:
"""
  solveBellman(agent::Agent,V0,ϵ=1e-6)

Solves the bellman equation for a given guess of V0 using iterateBellman
"""
function solveBellman(agent::Agent,V0,ϵ=1e-6)
  Vcont = V0
  a_policy = similar(V0,Int)
  diff = 1.

  while diff > ϵ
    V,a_policy = iterateBellman(agent,Vcont)
    diff = norm((V-Vcont)[:],Inf)
    Vcont = V
  end
  return Vcont,a_policy
end;

In [11]:
"""
  solveBellman(agent::Agent,V0,ϵ=1e-6)

Solves the bellman equation for a given guess of V0 using iterateBellman_alt
"""
function solveBellman_alt(agent::Agent,V0,ϵ=1e-6)
  Vcont = V0
  a_policy = similar(V0,Int)
  diff = 1.

  while diff > ϵ
    V,a_policy = iterateBellman_alt(agent,Vcont)
    diff = norm((V-Vcont)[:],Inf)
    Vcont = V
  end
  return Vcont,a_policy
end;

In [12]:
function constructTransitionMatrix(agent::Agent,a_policy)
  @unpack π,agrid = agent

  N = length(agrid)
  S = length(π)

  H = spzeros(N*S,N*S) #sparse matrix to save on space
  for n_ in 1:N
    for s in 1:S
      i_ = n_+N*(s-1) # index for ia_,s
      n = a_policy[n_,s]
      for sprime in 1:S
        i = n+N*(sprime-1)
        #probability of transitioning from state ia_,s to ia,sprime
        H[i_,i] = π[sprime]
      end
    end
  end
  return H
end;

In [13]:
function find_stationary_distribution(agent::Agent,a_policy)
  H = constructTransitionMatrix(agent,a_policy)
  N = size(H)[1]
  π0 = ones(1,N)/N
  diff = 1.
  while diff > 1e-10
    π1 = π0*H
    diff = norm(π1-π0,Inf)
    π0 = π1
  end

  return π0
end;

In [14]:
"""
  computeassetdemand(agent::Agent,r)

Computes asset demanded in steady state at interest rate r and
wage w
"""
function computeassetdemand(agent::Agent,r,w)
  agent.r = r
  agent.w = w

  N = length(agent.agrid)
  S = length(agent.e)
  V0 = zeros(N,S)

  V,a_policy = solveBellman_alt(agent,V0)
  λss = find_stationary_distribution(agent,a_policy)

  A = 0.
  for n in 1:N
    for s in 1:S
      i = n + (s-1)*N
      A  = A + λss[i] * agent.agrid[n]
    end
  end

  λss = reshape(λss,N,S) #λss[ia_,s] is the fraction of agents with assets agrid[ia]
  #and productivity e[s]

  return dot(agent.agrid,sum(λss,dims=2)) #agrid*πss gives average assets for each productivity
end;

In [20]:
agent = Agent(2.,0.96,.02,1.,[.8,1.,1.2],ones(3)/3,LinRange(-.5,25.,1000))
computeassetdemand(agent,.03435,1.)

0.00042094784010208216